In [53]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [54]:
%%sql
drop table if exists SeqTbl;
CREATE TABLE SeqTbl
 (seq INTEGER PRIMARY KEY); 

INSERT INTO SeqTbl VALUES (1);
INSERT INTO SeqTbl VALUES (2);
INSERT INTO SeqTbl VALUES (4);
INSERT INTO SeqTbl VALUES (5);
INSERT INTO SeqTbl VALUES (6);
INSERT INTO SeqTbl VALUES (7);
INSERT INTO SeqTbl VALUES (8);
INSERT INTO SeqTbl VALUES (11);
INSERT INTO SeqTbl VALUES (12);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

## やりたいこと
+ 欠番を見つける

In [61]:
%%sql
-- exceptを使う
select generate_series(1,12) as seq
except
select seq
from seqtbl;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


seq
3
10
9


In [62]:
%%sql
-- not inを使う
select *
from (
    select generate_series(1,12) as seq
) as tmp
where seq not in (
    select seq
    from seqtbl
)

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


seq
3
9
10


In [63]:
%%sql
-- not existsを使う
select *
from (
    select generate_series(1,12) as seq
) as tmp
where not exists (
    select *
    from seqtbl as s
    where tmp.seq = s.seq
)

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


seq
3
9
10


In [66]:
%%sql
-- 外部結合を使う
select t.seq
from (
    select generate_series(1,12) as seq
) as t
left join seqtbl as s
on t.seq = s.seq
where s.seq is null

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


seq
3
9
10
